# Дополнительный проект

Каждый из имеющихся файлов содержит, как минимум, 4 колонки (поля): FILIAL_ID, SUBS_ID, PROD_ID, ACT_DTTM.  

Суть задачи в том, чтобы проверить подключения продуктов определенным пользователям, соединив файлы о продажах с логами по подключениям в системе.  

Особенности данных:  

- сотрудники телемаркетинга не всегда указывают полный id, если  
'id' нет в начале SUBS_ID, то нужно его добавить  
- поля в файлах могут быть расположены абсолютно случайным образом, но названия полей статичны  
- продажа не засчитывается, если отключение (END_DTTM) произошло меньше  
чем через 5 минут после подключения (START_DTTM)  
- если в файле с продажами встречается строка без указанного SUBS_ID, она пропускается  

Сохраните результат в датафрэйм с разделителем ;, содержащий корректные подключения.

In [2]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [176]:
empty_df = pd.DataFrame()
shape_0 = 0#счетчик для проверки того, что мы не потеряли строки при слиянии

path = 'd://Carpov_courses/projects/subsid_5'
files = os.listdir(path)#список всех файлов с данными

for file in files:
    file_path = path + '/' + file
    if file != 'prod_activations_logs.csv':
        
        df = pd.read_csv(file_path, sep=';')
        df_temp = pd.concat([empty_df, df], ignore_index=True)
        empty_df = df_temp
        
        shape_0 += df.shape[0]
        
        print(file)
        print(df.columns)
    else:
        df_logs = pd.read_csv(file_path, sep=';')
        print(file)
        print(df_logs.columns)

df = empty_df

shape_0 == df.shape[0]#если True, то ничего не потеряли

prod_activations_logs.csv
Index(['SUBS_ID', 'PROD_ID', 'START_DTTM', 'END_DTTM'], dtype='object')
tm_sales_1.csv
Index(['SUBS_ID', 'FILIAL_ID', 'PROD_ID', 'ACT_DTTM'], dtype='object')
tm_sales_2.csv
Index(['FILIAL_ID', 'SUBS_ID', 'PROD_ID', 'ACT_DTTM'], dtype='object')
tm_sales_3.csv
Index(['FILIAL_ID', 'PROD_ID', 'SUBS_ID', 'ACT_DTTM'], dtype='object')


True

In [177]:
df.dropna(inplace = True)#убарием пропущенные значения

In [178]:
#функция, которая добавляет id в идентификатор, если его нет
def id_adding(x):
    if 'id' in x:
        pass
    else:
        x = 'id' + x
    return x

In [179]:
df['SUBS_ID'] = df['SUBS_ID'].apply(id_adding)
df.head()

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,id7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
3,id5416547,3,1743,17-03-2020 10:17
5,id8362218,7,9879,05-03-2020 11:42


In [181]:
df_logs = df_logs.merge(df, how='inner', on=['SUBS_ID','PROD_ID'])
df_logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,FILIAL_ID,ACT_DTTM
0,id4651830,1954,20-03-2020 14:59,01-12-2020 00:00,1,20-03-2020 14:59
1,id7646509,6431,19-03-2020 13:00,19-03-2020 13:03,5,19-03-2020 13:00
2,id5416547,1743,17-03-2020 10:17,25-03-2020 11:00,3,17-03-2020 10:17
3,id2185490,3210,16-03-2020 16:28,01-12-2020 00:00,2,16-03-2020 16:28
4,id7642700,3020,15-03-2020 14:21,15-03-2020 23:42,6,15-03-2020 14:21


In [167]:
pd.to_datetime(df_logs['END_DTTM'][1])

Timestamp('2020-03-19 13:03:00')

In [182]:
#переводим в формат времени колонки с датой
df_logs[['START_DTTM','END_DTTM','ACT_DTTM']] = df_logs[['START_DTTM','END_DTTM','ACT_DTTM']] \
.apply(pd.to_datetime, format='%d-%m-%Y %H:%M')

In [183]:
df_logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,FILIAL_ID,ACT_DTTM
0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,1,2020-03-20 14:59:00
1,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00,5,2020-03-19 13:00:00
2,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,3,2020-03-17 10:17:00
3,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,2,2020-03-16 16:28:00
4,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,6,2020-03-15 14:21:00


In [170]:
df_logs['START_DTTM']

0   2020-03-20 14:59:00
1   2020-03-19 13:00:00
2   2020-03-17 10:17:00
3   2020-03-16 16:28:00
4   2020-03-15 14:21:00
Name: START_DTTM, dtype: datetime64[ns]

In [171]:
df_logs['END_DTTM']

0   2020-12-01 00:00:00
1   2020-03-19 13:03:00
2   2020-03-25 11:00:00
3   2020-12-01 00:00:00
4   2020-03-15 23:42:00
Name: END_DTTM, dtype: datetime64[ns]

In [184]:
df_logs['DELTA'] =  df_logs['END_DTTM'] - df_logs['START_DTTM']
df_logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,FILIAL_ID,ACT_DTTM,DELTA
0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,1,2020-03-20 14:59:00,255 days 09:01:00
1,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00,5,2020-03-19 13:00:00,0 days 00:03:00
2,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,3,2020-03-17 10:17:00,8 days 00:43:00
3,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,2,2020-03-16 16:28:00,259 days 07:32:00
4,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,6,2020-03-15 14:21:00,0 days 09:21:00


In [185]:
df_logs['TRUE'] = np.where(df_logs['DELTA'].dt.total_seconds()/60 > 5, True, False)

In [186]:
df_logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM,FILIAL_ID,ACT_DTTM,DELTA,TRUE
0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00,1,2020-03-20 14:59:00,255 days 09:01:00,True
1,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00,5,2020-03-19 13:00:00,0 days 00:03:00,False
2,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00,3,2020-03-17 10:17:00,8 days 00:43:00,True
3,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00,2,2020-03-16 16:28:00,259 days 07:32:00,True
4,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00,6,2020-03-15 14:21:00,0 days 09:21:00,True


In [192]:
ids = list(df_logs.query('TRUE != 0').sort_values('SUBS_ID')['SUBS_ID'])

In [193]:
ids

['id2185490', 'id4651830', 'id5416547', 'id7642700']

In [195]:
df_logs.to_csv('data_logs', sep=';')